In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas

In [ ]:
data = '../data/'
data_out = '../results/data_out/'
fig_folder = '../results/figs/'

# File content: 1) Data read + state imputation + 2) indentify and plot states with type of employment (3 quadrants) + 3) adjust industry scenario to state proportional to current employment + 4) plot on map + 5) Add occupations and plot on map

## Data read + state imputation: Takes ~20 minutes. Skip if already done

In [ ]:
elec_gen = ['221111', '221112', '221113', '221114', '221115', '221116', '221117', '221118']

In [ ]:
xwalk_bea_bls = pd.read_csv('additional data/BEA_71_BLS_NAICS_proportional_no_ownership.csv', index_col=0)

In [ ]:
occ_ind_state = pd.read_excel(data + "BLS/oes_research_2018_allsectors.xlsx", usecols=['area', 'area_title', 'naics', 
                                            'naics_title', 'occ code', 'occ title', 'tot_emp', 'emp_prse'])
occ_ind_state = occ_ind_state[occ_ind_state['occ code'] =='00-0000']
occ_ind_state.naics=occ_ind_state.naics.str.ljust(6, '0')

# all required industries are available
print(set(xwalk_bea_bls.columns) - set(occ_ind_state.naics.astype(str)))

occ_ind_state = occ_ind_state[occ_ind_state.naics.isin(list(xwalk_bea_bls.columns) + ['221100'])]
occ_ind_state.drop_duplicates(['area_title', 'naics'], inplace=True)

In [ ]:
ind5_6 = pd.read_excel(data + 'BLS/oesm18in4/oesm18in4/nat5d_6d_M2018_dl.xlsx')
ind4 =  pd.read_excel(data + 'BLS/oesm18in4/oesm18in4/nat4d_M2018_dl.xlsx')
ind3 =  pd.read_excel(data + 'BLS/oesm18in4/oesm18in4/nat3d_M2018_dl.xlsx')
ind2 =  pd.read_excel(data + 'BLS/oesm18in4/oesm18in4/natsector_M2018_dl.xlsx')

ind_tot = pd.concat([ind2, ind3, ind4, ind5_6])
ind_tot.NAICS=ind_tot.NAICS.astype(str)
ind_tot.NAICS=ind_tot.NAICS.str.ljust(6, '0')

# all required industries are available
print(set(xwalk_bea_bls.columns) - set(ind_tot.NAICS.astype(str)))

# select total employment for naics in our crosswalk
ind_tot = ind_tot[ind_tot.NAICS.isin(list(xwalk_bea_bls.columns) + ['221100'])]
ind_tot = ind_tot[ind_tot.OCC_CODE == '00-0000']
ind_tot.drop_duplicates('NAICS', inplace=True)

In [ ]:
occ_ind_state.tot_emp = pd.to_numeric(occ_ind_state.tot_emp, errors='coerce')

In [ ]:
tot_emp_state = occ_ind_state[['area', 'area_title', 'tot_emp']].groupby(['area', 'area_title']).sum()

In [ ]:
ind_state_imputed = pd.DataFrame(columns=occ_ind_state.columns)
for ind, title in ind_tot[['NAICS', 'NAICS_TITLE']].values:
    ind_tot_emp = ind_tot[ind_tot.NAICS == ind].TOT_EMP.sum()
    state_df = occ_ind_state[occ_ind_state.naics == ind]
    ind_tot_emp_state = state_df.tot_emp.sum()
    if ind_tot_emp > ind_tot_emp_state:
        print(ind)
        remainder = ind_tot_emp - ind_tot_emp_state
        tot_emp_state_remains = tot_emp_state[~(tot_emp_state.index.get_level_values(1).isin(state_df[~state_df.tot_emp.isna()].area_title))]
        remainder_key = ((tot_emp_state_remains / tot_emp_state_remains.sum()) * remainder).round(0)
        remainder_key.reset_index(inplace=True)
        remainder_key['naics'] = ind
        remainder_key['naics_title'] = title
        remainder_key['occ code'] = '00-0000'
        remainder_key['occ title'] = 'All Occupations'
        remainder_key['emp_prse'] = 0.0
        a = pd.concat([state_df[~state_df.tot_emp.isna()], remainder_key]).reset_index(drop=True)
        ind_state_imputed = pd.concat([ind_state_imputed, a]).reset_index(drop=True)
    else: #ind_tot_emp < ind_tot_emp_state
        remainder = ind_tot_emp_state - ind_tot_emp
        # scale down existing
        state_df.tot_emp *= (ind_tot_emp / ind_tot_emp_state)
        ind_state_imputed = pd.concat([ind_state_imputed, state_df]).reset_index(drop=True)

In [ ]:
# impute 221120
trans_dist = (ind_tot[ind_tot.NAICS == '221100'].TOT_EMP - ind_tot[ind_tot.NAICS.isin(elec_gen)].TOT_EMP.sum()).values[0]

ind_tot.loc[len(ind_tot.index)] = ['221120', 'Electric Power Transmission and Distribution', '00-0000', 'Industry Total',
                                   'total', trans_dist, 0.0, 100, 0,0,0,0,0,0,0,0,0,0,0,0,0,0, 'NaN', 'NaN']

ind_tot = ind_tot[~(ind_tot.NAICS == '221100')]

In [ ]:
left_over = 0
for state in set(ind_state_imputed.area_title):
    state_df = ind_state_imputed[ind_state_imputed.area_title == state]
    trans_dist = (state_df[state_df.naics == '221100'].tot_emp.sum() - \
                  pd.to_numeric(state_df[state_df.naics.isin(elec_gen)].\
                                tot_emp, errors='coerce').sum())
    if trans_dist < 0:
        left_over += trans_dist
        trans_dist = 0
    ind_state_imputed.loc[len(ind_state_imputed.index)] = [ind_state_imputed[ind_state_imputed.area_title == state].area.values.mean()
                                                           , state, '221120', 
                                                           'Electric Power Transmission and Distribution',
                                                           '00-0000', 'All Occupations', trans_dist, 0.0]
    
tot_sum = ind_state_imputed[ind_state_imputed.naics == '221120'].tot_emp.sum()
tot_ind = ind_tot[ind_tot.NAICS == '221120'].TOT_EMP.sum()
ind_state_imputed.loc[ind_state_imputed.naics == '221120', 'tot_emp'] *= tot_ind / tot_sum

In [ ]:
ind_state_imputed.tot_emp = ind_state_imputed.tot_emp.round(0)

In [ ]:
ind_state_imputed.to_csv(data_out + 'BLS_oes_research_allsectors_2018_imputed.csv')

## Find 3 quadrants of occupations in states

In [ ]:
ind_state_imputed = pd.read_csv(data_out + 'BLS_oes_research_allsectors_2018_imputed.csv', index_col=0)
ind_state_imputed = ind_state_imputed[~(ind_state_imputed.naics == 221100)]

In [ ]:
xwalk_bea_bls = pd.read_csv('additional data/BEA_71_BLS_NAICS_proportional_no_ownership.csv', index_col=0)
xwalk_bea_bls.columns = xwalk_bea_bls.columns.astype(int)

In [ ]:
# all industries in crosswalk and data
print(set(xwalk_bea_bls.columns) - set(ind_state_imputed.naics))
print(set(ind_state_imputed.naics) - set(xwalk_bea_bls.columns))

In [ ]:
elec_capital_smallcaps = {'Biomass Electric Power Generation': 'Biomass electric power generation',
 'Coal Electric Power Generation': 'Coal electric power generation',
 'Electric Bulk Power Transmission and Control': 'Electric bulk power transmission and control',
 'Electric Power Distribution': 'Electric power distribution',
 'Gas Electric Power Generation': 'Gas electric power generation',
 'Geothermal Electric Power Generation':  'Geothermal electric power generation',
 'Hydroelectric Power Generation': 'Hydroelectric power generation',
 'Natural Gas Distribution': '221200',
 'Nuclear Electric Power Generation': 'Nuclear electric power generation',
 'Other Electric Power Generation': 'Other electric power generation',
 'Solar Electric Power Generation': 'Solar electric power generation',
 'Water, Sewage and Other Systems': '221300',
 'Wind Electric Power Generation': 'Wind electric power generation'}

In [ ]:
# crosswalk to BEA industries
ind_state_emp = ind_state_imputed.pivot_table(values='tot_emp', index='naics', columns='area_title').fillna(0)
ind_state_emp = xwalk_bea_bls @ ind_state_emp

ind_state_prop = ind_state_emp.div(ind_state_emp.sum(axis=1), axis=0).fillna(0)
ind_state_prop.rename(index=elec_capital_smallcaps, inplace=True)

In [ ]:
# impact relative to reference scenario
impact_type = pd.read_csv('../data/omn/occ_archetypes_thresholds_relbase_2034_2038.csv', index_col=0)

In [ ]:
perm_boost = impact_type[impact_type['Permanent_boost_r0.01'] == 1].index.values
phase_out = impact_type[impact_type['Phase_out_r0.01'] == 1].index.values
temp_boost = impact_type[impact_type['Temporary_boost_r0.01'] == 1].index.values

In [ ]:
emp_profile = pd.read_csv(data_out + 'bi-partite_emp_sum_elec.csv', index_col=0)
emp_profile = emp_profile.divide(emp_profile.sum(axis=1), axis=0).fillna(0)

state_occ = (ind_state_emp.T @ emp_profile).T

calculate location quotient:$\dfrac{x_{ab} / x_a}{x_b / x}$

$$ \dfrac{\text{\# workers in state a phase out } / 
\text{ \# workers in state a} }{\text{\# workers in phase out } / \text{ \# workers}} $$

In [ ]:
employment_occ = state_occ.sum(axis=1)
employment_state = state_occ.sum(axis=0)
employment_tot = state_occ.sum().sum()

In [ ]:
state_occ_quotient = state_occ.div(employment_state, axis=1).div((employment_occ / employment_tot), axis=0)

In [ ]:
quadrant_4 = pd.DataFrame(state_occ_quotient.loc[perm_boost].mean())
quadrant_4.columns = ['perm_boost']

quadrant_4['temp_boost'] = state_occ_quotient.loc[temp_boost].mean()
quadrant_4['phase_out'] = state_occ_quotient.loc[phase_out].mean()

In [ ]:
states = geopandas.read_file('additional data/usa-states-census-2014/usa-states-census-2014.shp')
states.STATEFP = states.STATEFP.astype(int)
us_boundary_map = states.boundary.plot(figsize=(3, 3), color="Black", linewidth=.5)

quadrant_geo = states.merge(quadrant_4, left_on = 'NAME', right_index=True, how='outer').dropna()

In [ ]:
quadrant_geo.drop_duplicates(inplace=True)

In [ ]:
#ax = ind_geo.boundary.plot(figsize=(18, 12), color="Black", linewidth=.5)
for col in ['perm_boost', 'temp_boost', 'phase_out']:
    vmin, vmax = (0.5, 2)
    if col == 'phase_out':
        vmin, vmax = (0.5, 4)
    quadrant_geo.plot(column =col, cmap='inferno', figsize=(10, 10), linewidth=.5, vmin=vmin, vmax=vmax, legend=True, legend_kwds={'shrink': 0.4})
    plt.tight_layout()
plt.savefig('no_elec_growth_impact_rel_states.png', bbox_inches='tight')
# save svg
plt.savefig('no_elec_growth_impact_rel_states.svg', bbox_inches='tight')
# save pdf
plt.savefig('no_elec_growth_impact_rel_states.pdf', bbox_inches='tight')